In [6]:
import re
from math import log

def tokenize(text):
    file = open(text, "r", encoding="ISO-8859-1")
    lines = file.readlines()
    tokens = []
    for i in range(len(lines)):
        lines[i] = lines[i].lower()
        x = list(lines[i])
        x = x[:-2]
        x = [re.sub(r"\d+", "", z) for z in x]
        x = [re.sub(r"[,.?:;\"@#$%^&*()_|/+=<>\-\!\[\]]", "", z) for z in x]
        x = ['<s>']+x+['</s>']
        if '' in x:
            x.remove('')
        tokens.append(x)
    return lines

def bigram_dict(data):
    bigram_counts = {}
    indi_counts = {}
    for sentence in data:
        for letter_idx in range(len(sentence)-1):
            letter1 = sentence[letter_idx]
            letter2 = sentence[letter_idx + 1]
            if letter1 not in bigram_counts:
                bigram_counts[letter1] = {}
                bigram_counts[letter1][letter2] = 1
            else:
                if letter2 in bigram_counts[letter1]:
                    bigram_counts[letter1][letter2] += 1
                else:
                    bigram_counts[letter1][letter2] = 1
            if letter1 not in indi_counts:
                indi_counts[letter1] = 1
            else:
                indi_counts[letter1] += 1
    return indi_counts, bigram_counts

def bigram_prob(indi_counts, bigram_counts, letter, prevletter):
    if prevletter not in bigram_counts.keys():
        num = 0
    else:
        if letter not in bigram_counts[prevletter]:
            num = 0
        else:
            num = bigram_counts[prevletter][letter]
    if prevletter not in indi_counts:
        denom = 0
    else:
        denom = indi_counts[prevletter]
    vocab_size = len(indi_counts.keys())
    prob = log(num + 1 / (denom + vocab_size))
    return prob

def accuracy():
    file_out = open("letterLangId.out", "r", encoding="ISO-8859-1")
    output_lines = file_out.readlines()
    file_sol = open("LangId.sol", "r", encoding="ISO-8859-1")
    sol_lines = file_sol.readlines()
    total = len(output_lines)
    count = 0
    for idx in range(len(output_lines)):
        if output_lines[idx] == sol_lines[idx]:
            count += 1
    acc = (count / total)*100
    return acc, count, total

eng_tokens = tokenize("LangId.train.English")
french_tokens = tokenize("LangId.train.French")
it_tokens = tokenize("LangId.train.Italian")
eng_ic, eng_bc = bigram_dict(eng_tokens)
french_ic, french_bc = bigram_dict(french_tokens)
it_ic, it_bc = bigram_dict(it_tokens)
test_tokens = tokenize("LangId.test")
with open('letterLangId.out', 'w') as output:
    count = 1
    for sen in test_tokens:
        prob_eng = 0
        prob_fr = 0
        prob_it = 0
        for letter_idx in range(1, len(sen)):
            prob_eng += bigram_prob(eng_ic, eng_bc, sen[letter_idx], sen[letter_idx-1])
            prob_fr += bigram_prob(french_ic, french_bc, sen[letter_idx], sen[letter_idx-1])
            prob_it += bigram_prob(it_ic, it_bc, sen[letter_idx], sen[letter_idx-1])
        if prob_eng >= prob_fr and prob_eng >= prob_it:
            lang = "English"
        elif prob_fr >= prob_eng and prob_fr >= prob_it:
            lang = "French"
        else:
            lang = "Italian"
        out_text = str(count)+" "+lang+"\n"
        output.write(out_text)
        count+=1
accuracy, count, total = accuracy()
print("Accuracy of the model is", accuracy)
print(count,"sentences match the solution out of", total)


Accuracy of the model is 99.33333333333333
298 sentences match the solution out of 300
